In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [26]:
df_genres = pd.read_parquet(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM\parquet\df_dummies.parquet')
df_games = pd.read_parquet(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM\parquet\games_clean3.parquet')

In [27]:
df_genres

,id,360 Video,Accounting,Action,Adventure,Animation & Modeling,Anime,Arcade,Audio Production,Beat 'em up,...,Tutorial,Twin Stick Shooter,Utilities,VR,Video Production,Violent,War,Web Publishing,World War II,Zombies
0,761140,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,643980,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,670290,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,767400,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,773570,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,773640,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32131,733530,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32132,610660,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32133,658870,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
df_games

,id,app_name,developer,genres,price,release_date
0,761140,Lost Summoner Kitty,Kotoshiro,Action,4.99,2018
1,643980,Ironbound,Secret Level SRL,Indie,0.00,2018
2,670290,Real Pool 3D - Poolians,Poolians.com,Casual,0.00,2017
3,767400,弹炸人2222,彼岸领域,Action,0.99,2017
4,773570,Log Challenge,None,Action,2.99,None
...,...,...,...,...,...,...
32130,773640,Colony On Mars,"Nikita ""Ghost_RUS""",Casual,1.99,2018
32131,733530,LOGistICAL: South Africa,Sacada,Casual,4.99,2018
32132,610660,Russian Roads,Laush Dmitriy Sergeevich,Indie,1.99,2018
32133,658870,EXIT 2 - Directions,"xropi,stev3ns",Casual,4.99,2017


In [29]:
print(df_games['id'].dtype)
print(df_genres['id'].dtype)


object
object


In [30]:
#Convert 'id' column to str to facilitate dataframe merge
df_games['id'] = df_games['id'].astype(str)


In [31]:
df_merged = df_games.merge(df_genres, on='id', how='left')

features = ['release_date'] + list(df_genres.columns[1:])  

# Normalize 'release_year'
scaler = StandardScaler()
df_merged['release_date'] = scaler.fit_transform(df_merged[['release_date']])


df_final = df_merged[['id'] + features]
df_final= df_final.merge(df_games[['id', 'app_name']], on='id', how='left')

In [32]:
# Sample a subset of your data, for example, 50%
df_sampled = df_final.sample(frac=0.5, random_state=42)

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(df_sampled[features].fillna(0))
similarity_matrix = np.nan_to_num(similarity_matrix)

In [33]:
def recomendacion_juego_muestreado(game_id, top_n=5):
    # Obtener los IDs de los juegos muestreados
    ids_juegos_muestreados = df_sampled['id'].unique()
    
    # Verificar si el ID del juego está en los juegos muestreados
    if game_id not in ids_juegos_muestreados:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    # Obtener el índice del juego en los datos muestreados
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    
    # Verificar si se encontró el juego en los datos muestreados
    if not indice_juego:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]

    # Calcular los puntajes de similitud entre juegos
    puntajes_similitud = list(enumerate(similarity_matrix[indice_juego]))
    puntajes_similitud = sorted(puntajes_similitud, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos similares
    indices_juegos_similares = [i for i, puntaje in puntajes_similitud[1:top_n+1]]
    
    # Obtener los nombres de los juegos similares
    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()

    # Mensaje de recomendación
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares


In [34]:
test_game_id = "774276"
recommendations = recomendacion_juego_muestreado(test_game_id, top_n=5)

print("Recomendaciones para ID de juego", test_game_id)
for rec in recommendations:
    print(rec)

Recomendaciones para ID de juego 774276
Juegos recomendados basados en el ID del juego 774276 - Rocksmith® 2014 Edition – Remastered – Grateful Dead - “Casey Jones”:
Loading Screen Simulator
Imperi
Rocksmith® 2014 Edition – Remastered – Variety Song Pack XII
Mystery Riddles
Rocksmith® 2014 Edition – Remastered – Goldfinger - “Superman”
